# 목표

Conv based로 D 모델만 학습이 가능한지 확인.

1. 가상의 G 모델(CLSTMC, pretrained)로 testset에 대해 prediction들을 생성
2. 생성한 데이터를 npy 파일로 저장
3. D 모델에서 가짜와 진짜 데이터를 구분이 가능한지(예측 데이터와 실제 데이터에 실제로 차이가 있는지) 를 확인한다.

In [1]:
from multiprocessing import cpu_count
from pathlib import Path
from typing import List, Union, AnyStr
from math import pi
import pickle

import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn import model_selection
from tqdm import tqdm
import torch_optimizer
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import random
from torchvision.models.resnet import resnet18

import torch_burn as tb
import utils

In [2]:
tb.seed_everything(0)

In [3]:
EXPERIMENT_NAME = '1130-Train-D'
CHECKPOINT_DIR = Path('checkpoint', EXPERIMENT_NAME)
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

DS_X = Path('data/1130/X.npy')
DS_Y = Path('data/1130/Y.npy')
DS_PRED = Path('data/1130/pred.npy')

# 데이터셋 생성

In [4]:
X = np.load(DS_X, allow_pickle=True)
Y = np.load(DS_Y, allow_pickle=True)
pred = np.load(DS_PRED, allow_pickle=True)

In [5]:
X.shape, Y.shape, pred.shape

((10671, 120, 6), (10671, 120, 3), (10671, 120, 3))

In [6]:
np.random.randint(0, 2, (10, ), np.uint8)

array([0, 0, 1, 0, 1, 0, 0, 1, 1, 0], dtype=uint8)

In [7]:
class DetectorDataset(Dataset):
    def __init__(self, X, Y, pred):
        super(DetectorDataset, self).__init__()

        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)
        self.pred = torch.tensor(pred, dtype=torch.float32)

        assert self.X.shape[0] == self.Y.shape[0] == self.pred.shape[0]
        self.L = self.pred.shape[0]

    def __len__(self):
        return 2 * self.L

    def __getitem__(self, idx):
        b = idx >= self.L
        x = self.Y[idx - self.L] if b else self.pred[idx]
        y = torch.tensor([int(b)], dtype=torch.float32)
        return x, y

In [8]:
ds = DetectorDataset(X, Y, pred)

# 모델 생성

In [9]:
class ResBlock1d(nn.Module):
    expansion = 1
    
    def __init__(self, inchannels, channels, kernel_size, stride=1):
        super(ResBlock1d, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv1d(inchannels, channels * self.expansion, kernel_size, stride=stride, padding=kernel_size // 2),
            nn.BatchNorm1d(channels),
            nn.LeakyReLU(),
            nn.Conv1d(channels, channels * self.expansion, kernel_size, stride=stride, padding=kernel_size // 2),
            nn.BatchNorm1d(channels)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(inchannels, channels * self.expansion, 1, stride=stride),
            nn.BatchNorm1d(channels)
        )
        self.act = nn.LeakyReLU()

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        identity = self.conv2(identity)
        x += identity
        x = self.act(x)

        return x

In [10]:
class ConvDetector(nn.Module):
    def __init__(self, block, layers):
        super(ConvDetector, self).__init__()
        
        self.inchannels = 64
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(3, self.inchannels, 7, stride=2, padding=3, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512 * block.expansion, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.transpose(1, 2)  # B, S, 6 --> B, 6, S
        x = self.conv1(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.sigmoid(x)

        return x
    
    def _make_layer(self, block, channels, blocks, stride=1):
        layers = []
        layers.append(block(self.inchannels, channels, 3))
        self.inchannels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inchannels, channels, 3))
        
        return nn.Sequential(*layers)

In [11]:
model = ConvDetector(ResBlock1d, [2, 2, 2, 2]).cuda()

In [12]:
model.eval()
with torch.no_grad():
    x, y = ds[0]
    print(model(x.unsqueeze(0).cuda()).cpu(), y)
model.train()
print()

tensor([[0.5080]]) tensor([0.])



In [13]:
criterion = nn.BCELoss().cuda()
optimizer = torch_optimizer.RAdam(model.parameters())

In [14]:
metrics = [tb.metrics.ModuleMetric(criterion, 'loss')]

In [15]:
callbacks = [
    tb.callbacks.EarlyStopping(metrics[0]),
    tb.callbacks.LRDecaying(optimizer, metrics[0], patience=3),
    tb.callbacks.SaveCheckpoint({'model': model}, metrics[0], CHECKPOINT_DIR, 'best-ckpt.pth')
]

In [16]:
trainer = tb.Trainer(model, optimizer, metrics, callbacks, ncols=100)

In [17]:
trainer.fit(ds, train_valid_split=0.2, num_epochs=200, batch_size=256, shuffle=True, pin_memory=True)

[0002/0200] Train:   0%|                                                     | 0/67 [00:00<?, ?it/s]

Save checkpoint: val_loss decreased from inf to 0.00038109553176635764


[0003/0200] Train:   0%|                                                     | 0/67 [00:00<?, ?it/s]

Save checkpoint: val_loss decreased from 0.00038109553176635764 to 0.0002047026774722754


[0004/0200] Train:   0%|                                                     | 0/67 [00:00<?, ?it/s]

...

val_loss is not improved for 1 epochs


[0043/0200] Train:   0%|                                                     | 0/67 [00:00<?, ?it/s]

Save checkpoint: val_loss decreased from 1.5859707486122396e-06 to 1.4402376978871281e-06


[0043/0200] Train:  55%|█████████████████              | 37/67 [00:02<00:01, 15.63it/s, loss 0.0000]


KeyboardInterrupt: 